In [1]:
import pandas as pd

In [18]:
import numpy as np
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sys import stderr
import pandas as pd
df = pd.read_csv('main_hr.csv')

try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    print('Downloading language model for the spaCy POS tagger\n'
        "(don't worry, this will only happen once)", file=stderr)
    from spacy.cli import download
    download('en')
    nlp = spacy.load('enen_core_web_sm')

# Load spaCy's English model with word vectors
# nlp = spacy.load("en")


def calculate_word_vectors(text):
    doc = nlp(text)
    # Average the word vectors for all tokens in the document
    return np.mean([token.vector for token in doc if not token.is_stop and not token.is_punct], axis=0)


user_prompt = "i want info related web developement and integrating into data science"
user_prompt_doc = nlp(user_prompt)

# Concatenate 'position' and 'tags' columns
features = df[['name', 'position', 'tags','contact','years_exp']].copy()
features['combined_text'] = features['name'] + ' ' + features['position'] + ' ' + features['tags']

# Calculate word vectors for combined text
features['word_vectors'] = features['combined_text'].apply(
    lambda x: calculate_word_vectors(str(x)))

# Calculate cosine similarity between user prompt and each row
features['cosine_similarity'] = features['word_vectors'].apply(
    lambda x: cosine_similarity([x], [user_prompt_doc.vector])[0][0])

# Sort the DataFrame by cosine similarity scores
features_sorted = features.sort_values(by='cosine_similarity', ascending=False)

# Add a 'rank' column to indicate the rank of each recommendation
features_sorted['rank'] = range(1, len(features_sorted) + 1)

# Recommend top 5 columns
top_5_recommendations = features_sorted.head(5)

# Display top 5 recommendations with ranks
print(top_5_recommendations[['rank','name', 'contact','position','years_exp']])


     rank      name     contact    position  years_exp
194     1   Motlagh  1009030917  technician          4
117     2     Goyal  6787601081  technician          5
43      3      Chan  1864106608  technician          4
64      4       Dee  6150492115  technician          4
69      5  Desimone  4704878143  technician          2


In [39]:
top_5_recommendations.reset_index(drop=True, inplace=True)

In [40]:
top_5_recommendations

,name,position,tags,contact,years_exp,combined_text,word_vectors,cosine_similarity,rank
0,Motlagh,technician,technic support troubleshoot instal repair mai...,1009030917,4,Motlagh technician technic support troubleshoo...,"[-0.14054212, -0.6373889, -0.057280555, 0.1045...",0.580506,1
1,Goyal,technician,technic support troubleshoot instal repair mai...,6787601081,5,Goyal technician technic support troubleshoot ...,"[-0.048434533, -0.62331563, 0.028899716, 0.114...",0.579648,2
2,Chan,technician,technic support troubleshoot instal repair mai...,1864106608,4,Chan technician technic support troubleshoot i...,"[-0.1550504, -0.6147052, 0.0038134933, 0.06908...",0.569325,3
3,Dee,technician,technic support troubleshoot instal repair mai...,6150492115,4,Dee technician technic support troubleshoot in...,"[-0.13142012, -0.57194346, -0.017912209, 0.049...",0.564049,4
4,Desimone,technician,technic support troubleshoot instal repair mai...,4704878143,2,Desimone technician technic support troublesho...,"[-0.13583252, -0.590025, -0.0015942322, 0.0625...",0.563958,5


In [48]:
send_data = []

for i in range(0,5):
    send_data.append(
        {
            "name" : top_5_recommendations['name'][i],
            "position" : top_5_recommendations['position'][i],
            "contact" : top_5_recommendations['contact'][i],
            "yoe" : top_5_recommendations['years_exp'][i]
        }
    )

In [51]:
send_data

[{'name': 'Motlagh',
  'position': 'technician',
  'contact': 1009030917,
  'yoe': 4},
 {'name': 'Goyal', 'position': 'technician', 'contact': 6787601081, 'yoe': 5},
 {'name': 'Chan', 'position': 'technician', 'contact': 1864106608, 'yoe': 4},
 {'name': 'Dee', 'position': 'technician', 'contact': 6150492115, 'yoe': 4},
 {'name': 'Desimone',
  'position': 'technician',
  'contact': 4704878143,
  'yoe': 2}]